# 📊 Notebook 04: Validación y Métricas de Rendimiento

Este notebook recopila **automáticamente** las métricas de los notebooks anteriores y ejecuta las consultas analíticas.

**Prerrequisitos:**
- Ejecutar `02_generador_datos.ipynb` (guarda métricas de ingesta)
- Ejecutar `03_etl_spark.ipynb` (guarda métricas de ETL)
- Los notebooks anteriores guardan sus tiempos en `metricas.json` (mismo directorio)

## 1. Configuración e Importaciones

In [ ]:
import time
import json
import os
from datetime import datetime

# Archivo de métricas compartido (en el mismo directorio de notebooks)
METRICS_FILE = 'metricas.json'

print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Cargar Métricas de Notebooks Anteriores (AUTOMÁTICO)

In [ ]:
# =====================================================
# 📥 CARGAR MÉTRICAS AUTOMÁTICAMENTE
# =====================================================

if os.path.exists(METRICS_FILE):
    with open(METRICS_FILE, 'r') as f:
        metricas = json.load(f)
    print("✅ Métricas cargadas automáticamente desde notebooks anteriores")
    print(f"\n📋 Contenido de {METRICS_FILE}:")
    print(json.dumps(metricas, indent=2))
else:
    print("⚠️ No se encontró el archivo de métricas.")
    print("   Ejecuta primero los notebooks 02 y 03.")
    metricas = {}

# Extraer valores para usar en el resumen
tiempo_ingesta = metricas.get('ingesta_cassandra', {}).get('tiempo_segundos', 0)
tiempo_etl = metricas.get('etl_spark', {}).get('tiempo_total', 0)

print(f"\n🕐 Tiempos extraídos:")
print(f"   - Ingesta Cassandra: {tiempo_ingesta:.2f} seg")
print(f"   - ETL Spark Total:   {tiempo_etl:.2f} seg")

## 3. Conexión a ClickHouse

In [ ]:
# Instalar cliente si no existe
!pip install -q clickhouse-connect

import clickhouse_connect

# Conectar a ClickHouse
client = clickhouse_connect.get_client(
    host='clickhouse',
    port=8123,
    database='dw_analitico'
)

# Verificar conexión
result = client.query("SELECT COUNT(*) as total FROM ventas_resumen")
total_registros = result.result_rows[0][0]
print(f"✅ Conectado a ClickHouse")
print(f"   - Registros en ventas_resumen: {total_registros:,}")

## 4. Consulta 1: Top 10 Categorías por Volumen de Ventas

In [ ]:
query_top10 = """
SELECT 
    categoria,
    sum(ventas_totales) as total_ventas_periodo
FROM dw_analitico.ventas_resumen
GROUP BY categoria
ORDER BY total_ventas_periodo DESC
LIMIT 10
"""

print("--- Ejecutando Consulta Top 10 ---")
start_time = time.time()

result_top10 = client.query(query_top10)

end_time = time.time()
tiempo_consulta_top10 = end_time - start_time

print(f"⏱️ Tiempo de ejecución: {tiempo_consulta_top10:.4f} segundos")
print("\n📊 Resultados:")
print("-" * 40)
for row in result_top10.result_rows:
    print(f"  {row[0]}: ${row[1]:,.2f}")

## 5. Consulta 2: Promedio de Ventas Diarias por Categoría

In [ ]:
query_promedio = """
SELECT 
    categoria,
    avg(ventas_totales) as promedio_ventas_diarias,
    sum(cantidad_transacciones) as transacciones_totales
FROM dw_analitico.ventas_resumen
GROUP BY categoria
ORDER BY categoria
"""

print("--- Ejecutando Consulta Promedio Diario ---")
start_time = time.time()

result_promedio = client.query(query_promedio)

end_time = time.time()
tiempo_consulta_promedio = end_time - start_time

print(f"⏱️ Tiempo de ejecución: {tiempo_consulta_promedio:.4f} segundos")
print("\n📊 Resultados:")
print("-" * 60)
for row in result_promedio.result_rows:
    print(f"  {row[0]}: Promedio ${row[1]:,.2f} | Transacciones: {row[2]:,}")

## 6. 📋 RESUMEN FINAL DE MÉTRICAS (PARA EL INFORME)

In [ ]:
import platform

print("="*70)
print("📊 RESUMEN DE MÉTRICAS DE RENDIMIENTO - PROYECTO DATA PIPELINE")
print("="*70)
print(f"\n🖥️ Entorno de Ejecución:")
print(f"   - Plataforma: Docker Container")
print(f"   - Python: {platform.python_version()}")
print(f"   - Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Objetivos
obj_ingesta = 300  # 5 minutos
obj_etl = 120  # 2 minutos
obj_query = 3  # 3 segundos

print("\n" + "-"*70)
print("\n📈 TABLA DE TIEMPOS (Copiar al informe):")
print("\n| Operación                  | Tiempo Real | Objetivo    | Cumple |")
print("|:---------------------------|:-----------:|:-----------:|:------:|")

# Ingesta
cumple_ing = "✅" if tiempo_ingesta < obj_ingesta else "❌"
print(f"| Ingesta Cassandra (100k)   | {tiempo_ingesta:>8.2f} s  | < 5 min     | {cumple_ing}     |")

# ETL
cumple_etl = "✅" if tiempo_etl < obj_etl else "❌"
print(f"| Transformación Spark ETL   | {tiempo_etl:>8.2f} s  | < 2 min     | {cumple_etl}     |")

# Consulta Top 10
cumple_q1 = "✅" if tiempo_consulta_top10 < obj_query else "❌"
print(f"| Consulta Top 10 (CH)       | {tiempo_consulta_top10:>8.4f} s  | < 3 seg     | {cumple_q1}     |")

# Consulta Promedio
cumple_q2 = "✅" if tiempo_consulta_promedio < obj_query else "❌"
print(f"| Consulta Promedio (CH)     | {tiempo_consulta_promedio:>8.4f} s  | < 3 seg     | {cumple_q2}     |")

print("\n" + "="*70)
print("💡 CONCLUSIÓN:")
todos_cumplen = all([
    tiempo_ingesta < obj_ingesta, 
    tiempo_etl < obj_etl, 
    tiempo_consulta_top10 < obj_query, 
    tiempo_consulta_promedio < obj_query
])
if todos_cumplen:
    print("   ✅ TODOS los objetivos de rendimiento fueron CUMPLIDOS.")
else:
    print("   ⚠️ Algunos objetivos NO fueron cumplidos. Revisar configuración.")
print("="*70)

## 7. Exportar Métricas Finales a Markdown

In [ ]:
# Actualizar archivo de métricas con consultas
metricas['consultas_clickhouse'] = {
    'consulta_top10': round(tiempo_consulta_top10, 4),
    'consulta_promedio': round(tiempo_consulta_promedio, 4),
    'registros_analizados': total_registros,
    'timestamp': datetime.now().isoformat()
}

with open(METRICS_FILE, 'w') as f:
    json.dump(metricas, f, indent=2)

# Generar archivo Markdown para el informe (en el mismo directorio)
output_md = 'metricas_reales.md'

with open(output_md, 'w', encoding='utf-8') as f:
    f.write("# 📊 Métricas de Rendimiento - Valores Reales\n\n")
    f.write(f"**Fecha de ejecución:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write("## Tabla Comparativa\n\n")
    f.write("| Operación | Tiempo Real | Objetivo | Cumple |\n")
    f.write("|:---|:---:|:---:|:---:|\n")
    f.write(f"| Ingesta Cassandra (100k) | {tiempo_ingesta:.2f} s | < 5 min | {'✅' if tiempo_ingesta < 300 else '❌'} |\n")
    f.write(f"| Transformación Spark ETL | {tiempo_etl:.2f} s | < 2 min | {'✅' if tiempo_etl < 120 else '❌'} |\n")
    f.write(f"| Consulta Top 10 (ClickHouse) | {tiempo_consulta_top10:.4f} s | < 3 seg | {'✅' if tiempo_consulta_top10 < 3 else '❌'} |\n")
    f.write(f"| Consulta Promedio (ClickHouse) | {tiempo_consulta_promedio:.4f} s | < 3 seg | {'✅' if tiempo_consulta_promedio < 3 else '❌'} |\n")
    f.write("\n## Detalles de Ejecución\n\n")
    f.write(f"- **Registros insertados en Cassandra:** {metricas.get('ingesta_cassandra', {}).get('registros', 'N/A'):,}\n")
    f.write(f"- **Registros procesados por Spark:** {metricas.get('etl_spark', {}).get('registros_entrada', 'N/A'):,}\n")
    f.write(f"- **Registros en ClickHouse:** {total_registros:,}\n")

print(f"✅ Métricas exportadas a:")
print(f"   - JSON: {METRICS_FILE}")
print(f"   - Markdown: {output_md}")
print(f"\n📝 Puedes copiar {output_md} a la carpeta docs/ para incluirlo en el informe.")